<a href="https://colab.research.google.com/github/inceatakan/SarcasmNewsHeadlines/blob/master/Sarcasm_Detection_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Detection in News Headlines Using BERT

In this notebook, we build a binary classifier to classify news headlines as 'sarcastic' or 'non-sarcastic' using [BERT](https://arxiv.org/abs/1810.04805), a pretrained NLP model that can be used for transfer learning on text data.  We will use the [*ktrain* library](https://github.com/amaiya/ktrain), a lightweight wrapper around Keras to help train (and deploy) neural networks.  For more information on *ktrain*, see [this Medium post](https://towardsdatascience.com/ktrain-a-lightweight-wrapper-for-keras-to-help-train-neural-networks-82851ba889c).

The News Headline dataset can be found: https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection 
We will begin by installing *ktrain* and importing the required *ktrain* modules.

In [0]:
# install ktrain
!pip3 install ktrain

In [0]:
# import ktrain
import ktrain
from ktrain import text

In [0]:
ktrain.__version__

'0.14.5'

Let's read Sarcasm data from Kaggle

In [0]:
import json
from google.colab import files
uploaded = files.upload()

Saving Sarcasm_Headlines_Dataset_v2.json to Sarcasm_Headlines_Dataset_v2 (3).json


In [0]:
# Reading data file
import pandas as pd

df = pd.read_json("Sarcasm_Headlines_Dataset_v2 (3).json", lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [0]:
# Dropping the column 'article_link'

df = df.drop(['article_link'], axis=1)
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [0]:
import numpy as np

X = df['headline']
y = df['is_sarcastic']

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
from ktrain import text as txt

MODEL_NAME = 'bert-base-uncased'
#MODEL_NAME = 'bert-large-cased'
t = text.Transformer(MODEL_NAME, maxlen=45,class_names=[0,1])
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_train, y_train)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=256)

preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 19


In [0]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Train for 90 steps, validate for 716 steps
Epoch 1/2
90/90 [==============================] - 989s 11s/step - loss: 0.4854 - accuracy: 0.7582 - val_loss: 0.2263 - val_accuracy: 0.9131
Epoch 2/2
90/90 [==============================] - 971s 11s/step - loss: 0.2149 - accuracy: 0.9150 - val_loss: 0.1479 - val_accuracy: 0.9447
